In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt # Visualization
import seaborn as sns # Visualization



# Data Preparation

---

## Data Extraction

In [ ]:
sample_submission = pd.read_csv("/kaggle/input/tabular-playground-series-nov-2021/sample_submission.csv")
train = pd.read_csv("/kaggle/input/tabular-playground-series-nov-2021/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-nov-2021/test.csv")

In [ ]:
sample_submission.shape

In [ ]:
train.shape

In [ ]:
train.info()

In [ ]:
test.shape

In [ ]:
test.info()

## Data Concatenation

In [ ]:
data = pd.concat([train, test], sort = False)
data.shape

In [ ]:
data.info()

In [ ]:
data.head()

## Null Check

In [ ]:
null_cols = [col for col in data.iloc[: , : -1].columns if data[col].isnull().sum() != 0]
null_cols

In [ ]:
float_cols = [col for col in data.iloc[: , 1 : -1].columns if data[col].dtype == "float64"]
len(float_cols)

In [ ]:
data.columns

In [ ]:
fig, axes = plt.subplots(20, 5, figsize = (16, 36))
axes = axes.flatten()

for idx, ax in enumerate(axes):
    
    sns.kdeplot(
        data=train.loc[: , 'f0':'target'], ax=ax, hue='target', fill=True,
        x=f'f{idx}', palette=['#4DB6AC', 'red'], legend=idx==0
    )
 
    ax.set_xticks([]); ax.set_yticks([]); ax.set_xlabel('')
    ax.set_ylabel(''); ax.spines['left'].set_visible(False)
    ax.set_title(f'f{idx}', loc='right', weight='bold', fontsize=10)

fig.supxlabel('Probability Density Function Estimation', ha='center', fontweight='bold')
fig.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(20, 5, figsize = (16, 36))
axes = axes.flatten()

for idx, ax in enumerate(axes):
    print(idx)
    print(ax)

In [ ]:
data.loc[: , 'f0':'f99'].head()